This script is used to calculate the emissions from fertilizer application and pesticides.

In [1]:
import pandas as pd
import bw2data as bd


In [5]:
"""Function to calculate emissions from fertilizer application."""
def calculate_fert_emissions(ui_Nitrogenous_fert_kg, ui_Phosphate_type_fert_kg):
    """Create dataframe for generic fertilizer content"""
    data_fertilizer_content = {
        'fertilizer_type': ['Ammonium Sulphate', 'Ammonium chloride', 'Calcium Ammonium Nitrate', 
                            'Calcium Nitrate', 'Urea', 
                            'Single superphosphate SSP 14%', 'Rock Phosphate (powder/granular)', 
                            'Potassium chloride (powder/granular)', 'Potassium Sulphate', 
                            'NPK 15-15-15', 'NPK 10-26-26'],
        'fert_N_content_%': [0.206, 0.250, 0.260, 0.155, 0.460, 0.000, 0.000, 0.000, 0.000, 0.150, 0.100],
        'fert_P2O5_content_%': [0.000, 0.000, 0.000, 0.000, 0.000, 0.140, 0.180, 0.000, 0.000, 0.150, 0.221],
        'fert_K2O_content_%': [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.60, 0.50, 0.15, 0.26]
    }
    df_fertilizer_content = pd.DataFrame(data_fertilizer_content)

    def calculate_individual_nutrient_contents(user_input_fert_type, user_input_fert_quantity_kg):
        fert_row = df_fertilizer_content[df_fertilizer_content['fertilizer_type'] == user_input_fert_type]
        
        if fert_row.empty:
            raise ValueError(f"Fertilizer type '{user_input_fert_type}' not found in the DataFrame.")
        
        nutrient_content_N = fert_row['fert_N_content_%'].values[0]
        nutrient_content_P = fert_row['fert_P2O5_content_%'].values[0]
        #nutrient_content_K = fert_row['fert_K2O_content_%'].values[0] #not needed for emissions
        
        ui_fert_N_content_kg = user_input_fert_quantity_kg * nutrient_content_N
        ui_fert_P_content_kg = user_input_fert_quantity_kg * nutrient_content_P
    
        return ui_fert_N_content_kg, ui_fert_P_content_kg

    """Calculate nutrient contents"""
    ui_fert_N_content_kg, ui_fert_P_content_kg = calculate_individual_nutrient_contents('Urea', ui_Nitrogenous_fert_kg)
    _, ui_fert_P_content_kg = calculate_individual_nutrient_contents('Single superphosphate SSP 14%', ui_Phosphate_type_fert_kg)

    """Emission factors for fertilizer application from PEFCR"""
    EF_N2O_fert = 0.022
    EF_NH3_syn_fert = 0.12
    EF_NO3_fert = 1.33
    EF_NOx_fert = 0.012
    EF_P_fert = 0.05

    Emission_kg_N2O = ui_fert_N_content_kg * EF_N2O_fert
    Emission_kg_NH3 = ui_fert_N_content_kg * EF_NH3_syn_fert
    Emission_kg_NO3 = ui_fert_N_content_kg * EF_NO3_fert
    Emission_kg_NOx = ((ui_fert_N_content_kg - Emission_kg_NH3 * 17 / 14) * EF_NOx_fert) * 46/14
    Emission_kg_P = ui_fert_P_content_kg * EF_P_fert
    
    emissions_data = {
        'Emission_kg_N2O': [Emission_kg_N2O],
        'Emission_kg_NH3': [Emission_kg_NH3],
        'Emission_kg_NO3': [Emission_kg_NO3],
        'Emission_kg_NOx_N': [Emission_kg_NOx],
        'Emission_kg_P': [Emission_kg_P]
    }
    
    df_emissions = pd.DataFrame(emissions_data)
    return df_emissions


#TESTING ONLY!!! DO NOT ADD TO MAIN CODE!!
print(calculate_fert_emissions(ui_Nitrogenous_fert_kg=1, ui_Phosphate_type_fert_kg=2))



   Emission_kg_N2O  Emission_kg_NH3  Emission_kg_NO3  Emission_kg_NOx_N  \
0          0.01012           0.0552           0.6118           0.003883   

   Emission_kg_P  
0          0.014  
<class 'pandas.core.frame.DataFrame'>
